<a href="https://colab.research.google.com/github/van1164/Pytorch_Study/blob/main/lab06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 10
learning_rate = 0.001

transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()
])


train_dataset = torchvision.datasets.CIFAR10(root='./',
                                             train=True,
                                             transform = transform,
                                             download =True)

test_dataset = torchvision.datasets.CIFAR10(root = './',
                                            train = False,
                                            transform =transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = 100,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size =100,
                                          shuffle =False)




100%|██████████| 170498071/170498071 [00:02<00:00, 62209988.70it/s]


Extracting ./cifar-10-python.tar.gz to ./


In [ ]:
class BasicBlock(nn.Module):
  def __init__(self, in_channels, out_channels, stride =1):
    super().__init__()

    self.residual_fuction = nn.Sequential(
        nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=stride,padding=1,bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.Conv2d(out_channels,out_channels,kernel_size=3,stride=1,padding=1,bias=False),
        nn.BatchNorm2d(out_channels),

    )

    self.shortcut = nn.Sequential()


    if stride !=1 or in_channels != out_channels:
      self.shortcut = nn.Sequential(
        nn.Conv2d(in_channels, out_channels,kernel_size=1,stride=stride,bias=False),
        nn.BatchNorm2d(out_channels)
      )

    self.relu = nn.ReLU()
    self.maxPool  = nn.MaxPool2d(4)
    self.fc = nn.Linear(out_channels*8*8,10)

  def forward(self,x):
    x = self.residual_fuction(x)+self.shortcut(x)
    x= self.relu(x)
    return x
model = BasicBlock(3,64)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
def update_lr(optimzer, lr):
  for param_group in optimizer.param_groups:
    param_group['lr']=lr

total_step = len(train_loader)
curr_lr = learning_rate

In [ ]:
def train(epoch):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        ouputs = model(images)
        loss = criterion(ouputs,labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1)%100 ==0:
            print("Epoch [{}/{}], step [{}/{}] Loss: {:4f}".format(epoch+1,num_epochs,i+1,total_step,loss.item()))
    
    if (epoch+1)%20 == 0:
        curr_lr /=3
        update_lr(optimizer,curr_lr)

def test():
    model.eval()
    with torch.no_grad():
        correct = 0
        total =0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data,1)
            total +=labels.size(0)
            correct +=(predicted == labels).sum().item()

        print('Accuracy of the model on the test images: {}%'.format(100*correct/total))


Batch
Normalization
외에
아래의
4
개
이상의
Normalization
방법에
대
해
찾아보고
,
구현한
결과를
보이시오

1. Batch Norm

In [ ]:
class ResNet(nn.Module):
  def __init__(self, block, num_block, num_classes =10, init_weights =True):
    super().__init__()

    self.in_channels=64

    self.conv1 = nn.Sequential(
      nn.Conv2d(3,64,kernel_size=7,stride=2,padding=3,bias=False),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
    )

    self.conv2_x = self._make_layer(block,64,num_block[0],1)
    self.conv3_x = self._make_layer(block,128,num_block[1],2)
    self.conv4_x = self._make_layer(block,256,num_block[2],2)
    self.conv5_x = self._make_layer(block,512,num_block[3],2)

    self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(512,num_classes)


  def _make_layer(self,block,out_channels,num_blocks,stride):
    strides = [stride]+[1]*(num_blocks-1)
    layer=[]
    for stride in strides :
      layer.append(block(self.in_channels,out_channels,stride))
      self.in_channels = out_channels

    return nn.Sequential(*layer)
  
  def forward(self, x):
    output = self.conv1(x)
    output = self.conv2_x(output)
    x = self.conv3_x(output)
    x = self.conv4_x(x)
    x= self.conv5_x(x)
    x = self.avg_pool(x)
    x=x.view(x.size(0),-1)
    x= self.fc(x)
    return x
model = ResNet(BasicBlock,[2,2,2,2])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
def update_lr(optimzer, lr):
  for param_group in optimizer.param_groups:
    param_group['lr']=lr

total_step = len(train_loader)
curr_lr = learning_rate

In [ ]:
for epoch in range(0,num_epochs):
  train(epoch)
  test()

KeyboardInterrupt: ignored

2. Layer Norm

In [ ]:
class ResNetL(nn.Module):
  def __init__(self, block, num_block, num_classes =10, init_weights =True):
    super().__init__()

    self.in_channels=64
    self.conv1 = nn.Sequential(
      nn.Conv2d(3,64,kernel_size=7,stride=2,padding=3,bias=False),
      nn.LayerNorm((self.in_channels,16,16)),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
    )

    self.conv2_x = self._make_layer(block,64,num_block[0],1)
    self.conv3_x = self._make_layer(block,128,num_block[1],2)
    self.conv4_x = self._make_layer(block,256,num_block[2],2)
    self.conv5_x = self._make_layer(block,512,num_block[3],2)

    self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(512,num_classes)


  def _make_layer(self,block,out_channels,num_blocks,stride):
    strides = [stride]+[1]*(num_blocks-1)
    layer=[]
    for stride in strides :
      layer.append(block(self.in_channels,out_channels,stride))
      self.in_channels = out_channels

    return nn.Sequential(*layer)
  
  def forward(self, x):
    output = self.conv1(x)
    output = self.conv2_x(output)
    x = self.conv3_x(output)
    x = self.conv4_x(x)
    x= self.conv5_x(x)
    x = self.avg_pool(x)
    x=x.view(x.size(0),-1)
    x= self.fc(x)
    return x
model = ResNetL(BasicBlock,[2,2,2,2])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
def update_lr(optimzer, lr):
  for param_group in optimizer.param_groups:
    param_group['lr']=lr

total_step = len(train_loader)
curr_lr = learning_rate

In [ ]:
for epoch in range(0,num_epochs):
  train(epoch)
  test()

InstanceNorm

In [ ]:
class ResNet(nn.Module):
  def __init__(self, block, num_block, num_classes =10, init_weights =True):
    super().__init__()

    self.in_channels=64

    self.conv1 = nn.Sequential(
      nn.Conv2d(3,64,kernel_size=7,stride=2,padding=3,bias=False),
      nn.InstanceNorm2d(64),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
    )

    self.conv2_x = self._make_layer(block,64,num_block[0],1)
    self.conv3_x = self._make_layer(block,128,num_block[1],2)
    self.conv4_x = self._make_layer(block,256,num_block[2],2)
    self.conv5_x = self._make_layer(block,512,num_block[3],2)

    self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(512,num_classes)


  def _make_layer(self,block,out_channels,num_blocks,stride):
    strides = [stride]+[1]*(num_blocks-1)
    layer=[]
    for stride in strides :
      layer.append(block(self.in_channels,out_channels,stride))
      self.in_channels = out_channels

    return nn.Sequential(*layer)
  
  def forward(self, x):
    output = self.conv1(x)
    output = self.conv2_x(output)
    x = self.conv3_x(output)
    x = self.conv4_x(x)
    x= self.conv5_x(x)
    x = self.avg_pool(x)
    x=x.view(x.size(0),-1)
    x= self.fc(x)
    return x
model = ResNet(BasicBlock,[2,2,2,2])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
def update_lr(optimzer, lr):
  for param_group in optimizer.param_groups:
    param_group['lr']=lr

total_step = len(train_loader)
curr_lr = learning_rate

In [ ]:
for epoch in range(0,num_epochs):
  train(epoch)
  test()

Epoch [1/10], step [100/500] Loss: 1.834224
Epoch [1/10], step [200/500] Loss: 1.522249
Epoch [1/10], step [300/500] Loss: 1.548688
Epoch [1/10], step [400/500] Loss: 1.430933
Epoch [1/10], step [500/500] Loss: 1.299627
Accuracy of the model on the test images: 51.44%
Epoch [2/10], step [100/500] Loss: 1.859818
Epoch [2/10], step [200/500] Loss: 1.523593
Epoch [2/10], step [300/500] Loss: 1.477445


KeyboardInterrupt: ignored

GroupNorm


In [ ]:
class ResNet(nn.Module):
  def __init__(self, block, num_block, num_classes =10, init_weights =True):
    super().__init__()

    self.in_channels=64

    self.conv1 = nn.Sequential(
      nn.Conv2d(3,64,kernel_size=7,stride=2,padding=3,bias=False),
      nn.GroupNorm(64,64),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
    )

    self.conv2_x = self._make_layer(block,64,num_block[0],1)
    self.conv3_x = self._make_layer(block,128,num_block[1],2)
    self.conv4_x = self._make_layer(block,256,num_block[2],2)
    self.conv5_x = self._make_layer(block,512,num_block[3],2)

    self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(512,num_classes)


  def _make_layer(self,block,out_channels,num_blocks,stride):
    strides = [stride]+[1]*(num_blocks-1)
    layer=[]
    for stride in strides :
      layer.append(block(self.in_channels,out_channels,stride))
      self.in_channels = out_channels

    return nn.Sequential(*layer)
  
  def forward(self, x):
    output = self.conv1(x)
    output = self.conv2_x(output)
    x = self.conv3_x(output)
    x = self.conv4_x(x)
    x= self.conv5_x(x)
    x = self.avg_pool(x)
    x=x.view(x.size(0),-1)
    x= self.fc(x)
    return x
model = ResNet(BasicBlock,[2,2,2,2])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
def update_lr(optimzer, lr):
  for param_group in optimizer.param_groups:
    param_group['lr']=lr

total_step = len(train_loader)
curr_lr = learning_rate

In [ ]:
for epoch in range(0,num_epochs):
  train(epoch)
  test()

Epoch [1/10], step [100/500] Loss: 1.653610
Epoch [1/10], step [200/500] Loss: 1.376367
Epoch [1/10], step [300/500] Loss: 1.764217


KeyboardInterrupt: ignored

ResNet34를 구현하시오.

In [18]:
class ResNet34(nn.Module):
  def __init__(self, block, num_block, num_classes =10, init_weights =True):
    super().__init__()

    self.in_channels=64
    self.conv1 =nn.Sequential(
      nn.Conv2d(3,64,kernel_size=7,stride=2,padding=3,bias=False),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
    )
    self.conv2_x = self._make_layer(block,64,num_block[0],3)
    self.conv3_x = self._make_layer(block,128,num_block[1],4)
    self.conv4_x = self._make_layer(block,256,num_block[2],6)
    self.conv5_x = self._make_layer(block,512,num_block[3],3)

    self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(512,num_classes)


  def _make_layer(self,block,out_channels,num_blocks,stride):
    strides = [stride]+[1]*(num_blocks-1)
    layer=[]
    for stride in strides :
      layer.append(block(self.in_channels,out_channels,stride))
      self.in_channels = out_channels

    return nn.Sequential(*layer)
  
  def forward(self, x):
    output = self.conv1(x)
    output = self.conv2_x(output)
    x = self.conv3_x(output)
    x = self.conv4_x(x)
    x= self.conv5_x(x)
    x = self.avg_pool(x)
    x=x.view(x.size(0),-1)
    x= self.fc(x)
    return x
model = ResNet34(BasicBlock,[2,2,2,2])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
def update_lr(optimzer, lr):
  for param_group in optimizer.param_groups:
    param_group['lr']=lr

total_step = len(train_loader)
curr_lr = learning_rate

In [ ]:
for epoch in range(0,num_epochs):
  train(epoch)
  test()

Epoch [1/10], step [100/500] Loss: 1.844864
Epoch [1/10], step [200/500] Loss: 1.575526
Epoch [1/10], step [300/500] Loss: 1.558331
Epoch [1/10], step [400/500] Loss: 1.424734
Epoch [1/10], step [500/500] Loss: 1.501102
Accuracy of the model on the test images: 49.91%
Epoch [2/10], step [100/500] Loss: 1.980172
Epoch [2/10], step [200/500] Loss: 1.546011
Epoch [2/10], step [300/500] Loss: 1.404390
Epoch [2/10], step [400/500] Loss: 1.484022
Epoch [2/10], step [500/500] Loss: 1.253393
Accuracy of the model on the test images: 51.97%
Epoch [3/10], step [100/500] Loss: 1.154403
Epoch [3/10], step [200/500] Loss: 1.659832
Epoch [3/10], step [300/500] Loss: 1.284458
Epoch [3/10], step [400/500] Loss: 1.341085
Epoch [3/10], step [500/500] Loss: 1.363340
Accuracy of the model on the test images: 55.71%
Epoch [4/10], step [100/500] Loss: 1.232064
Epoch [4/10], step [200/500] Loss: 1.359948
Epoch [4/10], step [300/500] Loss: 1.064928
Epoch [4/10], step [400/500] Loss: 1.394545
Epoch [4/10], ste

50
층
이상의
ResNet
은
매개변수
사용을
최소화
하기
위해
BottleNeck
을
이용한다
.
BottleNeck
을
구현하고
,
이를
이용하여
ResNet50
도
구현하시
오
(
구현
결과를
CIFAR10
에
학습
/
검증하시오
)

In [ ]:
class Bottleneck(nn.Module):
  def __init__(self, in_channels, out_channels, stride =1):
    super().__init__()

    self.residual_fuction = nn.Sequential(
        nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=stride,padding=1,bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.Conv2d(out_channels,out_channels,kernel_size=3,stride=1,padding=1,bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.Conv2d(out_channels,out_channels*4,kernel_size=1,stride=1,padding=1,bias=False),
        nn.BatchNorm2d(out_channels*4),
        nn.ReLU(),
        

    )

    self.shortcut = nn.Sequential()


    if stride !=1 or in_channels != out_channels:
      self.shortcut = nn.Sequential(
        nn.Conv2d(in_channels, out_channels*4,kernel_size=1,stride=stride,bias=False),
        nn.BatchNorm2d(out_channels*4)
      )

    self.relu = nn.ReLU()
    self.maxPool  = nn.MaxPool2d(4)
    self.fc = nn.Linear(out_channels*8*8,10)

  def forward(self,x):
    x = self.residual_fuction(x)+self.shortcut(x)
    x= self.relu(x)
    return x


In [ ]:
class ResNet50(nn.Module):
  def __init__(self, block, num_block, num_classes =10, init_weights =True):
    super().__init__()

    self.in_channels=64
    self.conv1 =nn.Sequential(
      nn.Conv2d(3,64,kernel_size=7,stride=2,padding=3,bias=False),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
    )
    self.conv2_x = self._make_layer(block,64,num_block[0],3)
    self.conv3_x = self._make_layer(block,128,num_block[1],4)
    self.conv4_x = self._make_layer(block,256,num_block[2],6)
    self.conv5_x = self._make_layer(block,512,num_block[3],3)

    self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(512,num_classes)


  def _make_layer(self,block,out_channels,num_blocks,stride):
    strides = [stride]+[1]*(num_blocks-1)
    layer=[]
    for stride in strides :
      layer.append(block(self.in_channels,out_channels,stride))
      self.in_channels = out_channels

    return nn.Sequential(*layer)
  
  def forward(self, x):
    output = self.conv1(x)
    output = self.conv2_x(output)
    x = self.conv3_x(output)
    x = self.conv4_x(x)
    x= self.conv5_x(x)
    x = self.avg_pool(x)
    x=x.view(x.size(0),-1)
    x= self.fc(x)
    return x
model = ResNet34(BasicBlock,[2,2,2,2])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
def update_lr(optimzer, lr):
  for param_group in optimizer.param_groups:
    param_group['lr']=lr

total_step = len(train_loader)
curr_lr = learning_rate

In [ ]:
for epoch in range(0,num_epochs):
  train(epoch)
  test()